In [ ]:
!pip install opencv-python

In [1]:
import tkinter as tk
from tkinter import ttk
import cv2
from PIL import Image, ImageTk

skip_frames = 5

class VideoPlayer:
    def __init__(self, video_path):
        self.video_path = video_path  # Save the path for reinitializing capture
        self.cap = cv2.VideoCapture(video_path)
        if not self.cap.isOpened():
            print(f"Error opening video file: {video_path}")
        self.playing = False
        self.current_frame = None
        self.frame_no = 0
        
        # Read the first frame to initialize
        ret, frame = self.cap.read()
        if ret:
            self.current_frame = frame
            self.frame_no = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
        else:
            print("Error reading first frame from:", video_path)

    def update(self):
        if self.playing:
            ret, frame = self.cap.read()
            if ret:
                self.current_frame = frame
                self.frame_no = int(self.cap.get(cv2.CAP_PROP_POS_FRAMES))
            else:
                # Video ended; loop back to beginning
                self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
                ret, frame = self.cap.read()
                if ret:
                    self.current_frame = frame
                    self.frame_no = 0
        return self.current_frame

    def toggle_playback(self):
        self.playing = not self.playing
        print("Video toggled; now", "playing" if self.playing else "paused")

    def set_playing(self, state):
        self.playing = state
        print("Set video playing to", state)

    def stop(self):
        self.playing = False
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        ret, frame = self.cap.read()
        if ret:
            self.current_frame = frame
            self.frame_no = 0
        print("Video stopped and reset.")

    def seek(self, frames):
        new_frame_no = max(self.frame_no + frames, 0)
        if frames < 0:
            # For backward seeking, reinitialize the capture.
            self.cap.release()
            self.cap = cv2.VideoCapture(self.video_path)
            count = 0
            frame = None
            # Read frames sequentially until reaching new_frame_no.
            while count <= new_frame_no:
                ret, frame = self.cap.read()
                if not ret:
                    print("Seek error: Unable to reach desired frame.")
                    return
                count += 1
            self.current_frame = frame
            self.frame_no = new_frame_no
            print(f"Seeked backward to frame {self.frame_no}")
        else:
            # For forward seeks, use set() directly.
            self.cap.set(cv2.CAP_PROP_POS_FRAMES, new_frame_no)
            ret, frame = self.cap.read()
            if ret:
                self.current_frame = frame
                self.frame_no = new_frame_no
                print(f"Seeked to frame {self.frame_no}")
            else:
                print("Seek error.")

class VideoApp:
    def __init__(self, root, video_paths, scale_factor=0.3):
        self.root = root
        self.root.title("Multiresponder Video Sync")
        self.scale_factor = scale_factor
        self.master_playing = False  # Master play state
        
        # Main container for video players
        self.container = ttk.Frame(root)
        self.container.pack(padx=10, pady=10)
        
        self.players = []
        self.panels = []
        self.frame_labels = []
        
        # Create a subframe for each video with its controls and frame display
        for idx, path in enumerate(video_paths):
            player = VideoPlayer(path)
            self.players.append(player)
            
            # Frame for each video and its controls
            frame = ttk.Frame(self.container, borderwidth=2, relief="sunken")
            frame.grid(row=0, column=idx, padx=5, pady=5)
            
            # Label to display the video
            panel = tk.Label(frame)
            panel.pack()
            self.panels.append(panel)
            
            # Label to display the current frame number
            frame_label = tk.Label(frame, text="Frame: 0")
            frame_label.pack(pady=2)
            self.frame_labels.append(frame_label)
            
            # Create video-specific control buttons
            ctrl_frame = ttk.Frame(frame)
            ctrl_frame.pack(pady=5)
            
            play_pause_btn = ttk.Button(ctrl_frame, text="Play/Pause",
                                        command=player.toggle_playback)
            play_pause_btn.pack(fill='x')
            
            stop_btn = ttk.Button(ctrl_frame, text="Stop", command=player.stop)
            stop_btn.pack(fill='x', pady=2)
            
            forward_btn = ttk.Button(ctrl_frame, text="Forward",
                                     command=lambda p=player: p.seek(skip_frames))
            forward_btn.pack(fill='x')
            
            backward_btn = ttk.Button(ctrl_frame, text="Backward",
                                      command=lambda p=player: p.seek(-skip_frames))
            backward_btn.pack(fill='x', pady=2)
        
        # Master control frame for global playback
        self.master_frame = ttk.Frame(root)
        self.master_frame.pack(pady=10)
        self.master_button = ttk.Button(self.master_frame, text="Master Play", command=self.master_toggle)
        self.master_button.pack()
        
        # Start the update loop
        self.update()

    def master_toggle(self):
        # Toggle the master play state and apply it to all players
        new_state = not self.master_playing
        self.master_playing = new_state
        if new_state:
            self.master_button.config(text="Master Pause")
        else:
            self.master_button.config(text="Master Play")
        for player in self.players:
            player.set_playing(new_state)

    def update(self):
        # Update each video frame and corresponding frame number label
        for idx, player in enumerate(self.players):
            frame = player.update()
            if frame is not None:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_resized = cv2.resize(frame_rgb, None, fx=self.scale_factor, fy=self.scale_factor)
                img = Image.fromarray(frame_resized)
                imgtk = ImageTk.PhotoImage(image=img)
                self.panels[idx].imgtk = imgtk  # prevent garbage collection
                self.panels[idx].configure(image=imgtk)
                self.frame_labels[idx].config(text=f"Frame: {player.frame_no}")
        self.root.after(30, self.update)


In [2]:
# List the three video file paths (adjust to your files)
video_paths = ["/standard/UVA-DSA/NIST EMS Project Data/DataCollection_Spring_2025/CARS/03-28/stroke/t1/GoPro/primary/GX010012.MP4", "/standard/UVA-DSA/NIST EMS Project Data/DataCollection_Spring_2025/CARS/03-28/stroke/t1/GoPro/secondary/GX010006.MP4", "/standard/UVA-DSA/NIST EMS Project Data/DataCollection_Spring_2025/CARS/03-28/stroke/t1/GoPro/driver/GX_COMBINED.mp4"]

root = tk.Tk()
app = VideoApp(root, video_paths, scale_factor=0.1)
root.mainloop()

TclError: no display name and no $DISPLAY environment variable